In [1]:
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
import numpy as np

In [ ]:
#Gathering players that participated in combine
firstLast=[]

dfNames = pd.read_csv("NBACombine.csv")

names = dfNames["Player"]

for i in names:
    j=i.split()
    firstLast.append([j[0].lower(),j[1].lower()])
print(firstLast)


'dfNames = pd.read_csv("NBACombine.csv")\n\nnames = dfNames["Player"]\n\nfor i in names:\n    j=i.split()\n    firstLast.append([j[0].lower(),j[1].lower()])\nprint(firstLast)'

Shortened version of code to go over the first 4 rows of combine participants, Full code took hours and was run in VSCode. This also filters out the international players that did not play in college.

In [ ]:
###GPT Test code

df2 = pd.DataFrame()
count=0

for i in firstLast:
    cont = True
    url = f'https://www.sports-reference.com/cbb/players/{i[0]}-{i[1]}-1.html'
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    table = soup.find('table', id='players_per_game')

    # Look in comments if table is hidden
    if table is None:
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        for comment in comments:
            if 'id="players_per_game"' in comment:
                comment_soup = BeautifulSoup(comment, 'html.parser')
                table = comment_soup.find('table', id='players_per_game')
                if table:
                    break

    if table is None:
        print("(no college stats table found)")
        continue  # skip this player

    # Read table into df
    df = pd.read_html(str(table))[0]

    # Drop completely empty rows
    df = df.dropna(how="all")

                            #ChatGPT
    # -------  KEY PART: keep only the Career row ------- #
    # Most cbb tables have a "Season" column with 'Career' in it
    if 'Season' in df.columns:
        career = df[df['Season'] == 'Career']
    else:
        # fallback: assume last row is career totals
        career = df.tail(1)
    # ----------------------------------------------------- #

    if career.empty:
        continue

    # Add name and append
    career["Name"] = i[0].capitalize() + ' ' + i[1].capitalize()
    df2 = pd.concat([df2, career], ignore_index=True)

    lag = np.random.uniform(low=5, high=30)
    print(f'...waiting {round(lag, 1)} seconds')
    time.sleep(lag)
    count += 1
df2


/var/folders/5h/_7p0899150l8hd4khcp86lf00000gn/T/ipykernel_28313/3749288345.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/var/folders/5h/_7p0899150l8hd4khcp86lf00000gn/T/ipykernel_28313/3749288345.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career["Name"] = i[0].capitalize() + ' ' + i[1].capitalize()


...waiting 24.9 seconds


/var/folders/5h/_7p0899150l8hd4khcp86lf00000gn/T/ipykernel_28313/3749288345.py:28: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/var/folders/5h/_7p0899150l8hd4khcp86lf00000gn/T/ipykernel_28313/3749288345.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career["Name"] = i[0].capitalize() + ' ' + i[1].capitalize()


...waiting 23.4 seconds


,Season,Team,Conf,Class,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Name
0,Career,Career,Career,Career,NaN,58,35,23.1,3.8,8.0,...,3.1,4.6,1.0,0.4,0.3,0.9,1.4,12.3,NaN,Semi Ojeleye
1,Career,Career,Career,Career,NaN,67,62,32.8,4.7,10.4,...,3.4,4.3,2.1,0.6,0.3,1.3,2.0,13.3,NaN,Rodney Hood


In [51]:
#Filter out irrelivant stats to quick clean. Also only career stats
df_new = df2.drop(columns=(["Season","Team","Conf","Class","Pos","Awards"]))
df_new

,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Name
0,58,35,23.1,3.8,8.0,.479,1.4,3.4,.415,2.4,...,1.6,3.1,4.6,1.0,0.4,0.3,0.9,1.4,12.3,Semi Ojeleye
1,67,62,32.8,4.7,10.4,.455,1.8,4.4,.396,3.0,...,0.9,3.4,4.3,2.1,0.6,0.3,1.3,2.0,13.3,Rodney Hood


In [14]:
df_new.to_csv("CollegeStats.csv",index=False)
df_new

,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Name
1,30.0,30.0,33.3,6.8,14.7,0.46,1.6,4.5,0.346,5.2,...,2.1,5.1,7.2,1.3,1.0,1.3,2.0,2.8,17.6,Airious Bailey
6,133.0,61.0,22.1,3.2,7.1,0.456,1.1,2.8,0.373,2.2,...,0.5,1.6,2.1,1.0,0.7,0.1,1.1,1.6,9.2,Tamar Bates
14,149.0,37.0,24.9,3.1,6.8,0.45,2.1,4.9,0.434,0.9,...,0.3,2.9,3.1,1.1,0.4,0.2,0.7,1.0,8.8,Koby Brea
22,168.0,163.0,27.2,6.5,12.1,0.539,0.4,1.2,0.302,6.1,...,3.1,6.4,9.5,1.7,0.8,2.5,1.8,2.4,16.1,Johni Broome
26,37.0,5.0,19.3,2.2,4.8,0.46,1.1,2.8,0.371,1.1,...,0.9,3.2,4.1,1.0,0.9,1.0,1.0,2.3,6.5,Carter Bryant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716,35,35,28.5,5.4,12.7,0.423,2.3,6.6,0.353,3.0,...,0.3,3.2,3.5,4.1,1.1,0.3,2.7,1.9,16.1,Coby White
1719,66,65,30.8,5.5,12.6,0.436,1.9,5.5,0.343,3.6,...,1.2,3.6,4.8,1.7,0.7,0.5,1.6,2.3,15.5,Kris Wilkes
1723,104,101,28.9,5.3,10.2,0.516,0.3,1.0,0.291,5.0,...,2.5,4.0,6.5,2.1,0.9,1.5,2.1,3.2,15.7,Grant Williams
1725,33,33,30.0,9.0,13.2,0.68,0.7,2.2,0.338,8.2,...,3.5,5.4,8.9,2.1,2.1,1.8,2.4,2.1,22.6,Zion Williamson
